<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/Preprocess_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mne matplotlib

     |████████████████████████████████| 7.5 MB 6.3 MB/s 


In [2]:
import numpy as np
import mne 
import pandas as pd
import os
import pylab as pl
import re

Uploading files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg

/content/drive/.shortcut-targets-by-id/1UmPRvBaUSHwpdFfssLmk_CxDx3urhuBi/isip_tuh_eeg


In [7]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg

Выходные данные были обрезаны до нескольких последних строк (5000).
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t002.edf
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t002.tse
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t003.edf
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t003.tse
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t004.edf
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t004.tse
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t005.edf
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t005.tse
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t006.edf
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t006.tse
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t009.edf
train__01_tcp_ar__104__00010480__s013_2013_07_22__00010480_s013_t009.tse
train__01_tcp_ar__104__00010480__s013_2013_07_22__000104

In [8]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm import tqdm
import shutil
from collections import Counter
import pandas as pd

In [12]:
drive_tuh_eeg_loc = '/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg'

Creating two different list for .tse and .edf files 

In [20]:
tse_files = list(Path(drive_tuh_eeg_loc).glob('*.tse'))

In [21]:
tse_files[:3]

[PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t003.tse'),
 PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t004.tse'),
 PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__02_tcp_le__058__00005804__s002_2008_11_05__00005804_s002_t005.tse')]

In [22]:
edf_files = list(Path(drive_tuh_eeg_loc).glob('*.edf'))

In [23]:
edf_files[:3]

[PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__03_tcp_ar_a__072__00007221__s003_2010_11_01__00007221_s003_t003.edf'),
 PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__03_tcp_ar_a__072__00007221__s003_2010_11_01__00007221_s003_t004.edf'),
 PosixPath('/content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__03_tcp_ar_a__072__00007234__s002_2010_11_24__00007234_s002_t000.edf')]

In [40]:
data_files = pd.DataFrame(
    [s.name.split('__') for s in edf_files[:10]],
    columns=[
      'type',
      2, #??
      3,
      4,
      5,
      'filename'
    ]
    )

In [35]:
data_files

,type,2,3,4,5,filename
0,train,03_tcp_ar_a,072,00007221,s003_2010_11_01,00007221_s003_t003.edf
1,train,03_tcp_ar_a,072,00007221,s003_2010_11_01,00007221_s003_t004.edf
2,train,03_tcp_ar_a,072,00007234,s002_2010_11_24,00007234_s002_t000.edf
3,train,03_tcp_ar_a,072,00007234,s002_2010_11_24,00007234_s002_t001.edf
4,train,03_tcp_ar_a,072,00007234,s002_2010_11_24,00007234_s002_t002.edf
5,train,03_tcp_ar_a,072,00007235,s004_2010_11_22,00007235_s004_t000.edf
6,train,03_tcp_ar_a,072,00007235,s004_2010_11_22,00007235_s004_t001.edf
7,train,03_tcp_ar_a,072,00007235,s004_2010_11_22,00007235_s004_t002.edf
8,train,03_tcp_ar_a,072,00007235,s005_2010_11_23,00007235_s005_t000.edf
9,train,03_tcp_ar_a,072,00007235,s005_2010_11_23,00007235_s005_t001.edf


In [38]:
channels = ['EEG FP1-REF',
              'EEG FP2-REF',
              'EEG F3-REF',
              'EEG F4-REF',
              'EEG C3-REF',
              'EEG C4-REF',
              'EEG P3-REF',
              'EEG P4-REF',
              'EEG O1-REF',
              'EEG O2-REF',
              'EEG F7-REF',
              'EEG F8-REF',
              'EEG T3-REF',
              'EEG T4-REF',
              'EEG T5-REF',
              'EEG T6-REF',
              'EEG FZ-REF',
              'EEG CZ-REF',
              'EEG PZ-REF',
              'EEG A1-REF',
              'EEG A2-REF']

In [41]:
#creating a separate column for each of the channels
for i in range(0,len(channels)):
  for name in channels:
    data_files[name] = 'NaN'

In [44]:
data_files.head()

,type,2,3,4,5,filename,EEG FP1-REF,EEG FP2-REF,EEG F3-REF,EEG F4-REF,...,EEG F8-REF,EEG T3-REF,EEG T4-REF,EEG T5-REF,EEG T6-REF,EEG FZ-REF,EEG CZ-REF,EEG PZ-REF,EEG A1-REF,EEG A2-REF
0,train,03_tcp_ar_a,072,00007221,s003_2010_11_01,00007221_s003_t003.edf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,train,03_tcp_ar_a,072,00007221,s003_2010_11_01,00007221_s003_t004.edf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,train,03_tcp_ar_a,072,00007234,s002_2010_11_24,00007234_s002_t000.edf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,train,03_tcp_ar_a,072,00007234,s002_2010_11_24,00007234_s002_t001.edf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,train,03_tcp_ar_a,072,00007234,s002_2010_11_24,00007234_s002_t002.edf,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
def preprocessing(file):
  #read the data
  data = mne.io.read_raw_edf(file)
  data = data.copy().load_data()
  #notch filter 60 Hz - to get rid of the noise because of the power outlets
  eeg_picks = mne.pick_types(data.info, eeg=True)
  freqs = []
  for i in range(0, int(data.info['sfreq']/2), 60):
    if i != 0:
      freqs.append(i)
  data_notch = data.notch_filter(freqs=np.asarray(freqs), picks=eeg_picks)

  #high pass filter
  data_filter = data_notch.filter(l_freq=0.2, h_freq=None)

  #downsampling
  data_done = data_filter.resample(sfreq=200)

  #channel reduction
  print(data.ch_names)
  kick_out = "yes"
  channels2drop = []
  for j in range(0, len(data.ch_names)):
    for i in range(0, len(channels)):
      if data.ch_names[j] == channels[i]:
        kick_out = "no"
        break
      else:
        kick_out = "yes"
    if kick_out == "yes":
      print(f"kick out {j}")
      channels2drop.append(j)
    else:
      print(f"keep {j}")
    names2drop = []
  for i in range(0,len(channels2drop)):
    names2drop.append(data.ch_names[channels2drop[i]])
  for drop in names2drop:
    data.drop_channels(data.ch_names[data.ch_names.index(drop)])

  return data_done

In [56]:
#PRACTICE
file = edf_files[0]
tryout = preprocessing(file)

Extracting EDF parameters from /content/drive/MyDrive/MLTSA22/Final_Project/isip_tuh_eeg/train__03_tcp_ar_a__072__00007221__s003_2010_11_01__00007221_s003_t003.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 100799  =      0.000 ...   251.998 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower

In [57]:
data_files.columns

Index([       'type',             2,             3,             4,
                   5,    'filename', 'EEG FP1-REF', 'EEG FP2-REF',
        'EEG F3-REF',  'EEG F4-REF',  'EEG C3-REF',  'EEG C4-REF',
        'EEG P3-REF',  'EEG P4-REF',  'EEG O1-REF',  'EEG O2-REF',
        'EEG F7-REF',  'EEG F8-REF',  'EEG T3-REF',  'EEG T4-REF',
        'EEG T5-REF',  'EEG T6-REF',  'EEG FZ-REF',  'EEG CZ-REF',
        'EEG PZ-REF',  'EEG A1-REF',  'EEG A2-REF'],
      dtype='object')

In [60]:
try_raw = tryout.get_data()

In [61]:
try_raw.shape

(19, 50400)

In [62]:
df = data_files.copy()

In [66]:
df.columns

Index([       'type',             2,             3,             4,
                   5,    'filename', 'EEG FP1-REF', 'EEG FP2-REF',
        'EEG F3-REF',  'EEG F4-REF',  'EEG C3-REF',  'EEG C4-REF',
        'EEG P3-REF',  'EEG P4-REF',  'EEG O1-REF',  'EEG O2-REF',
        'EEG F7-REF',  'EEG F8-REF',  'EEG T3-REF',  'EEG T4-REF',
        'EEG T5-REF',  'EEG T6-REF',  'EEG FZ-REF',  'EEG CZ-REF',
        'EEG PZ-REF',  'EEG A1-REF',  'EEG A2-REF'],
      dtype='object')

In [70]:
df.at[0, 'EEG FP1-REF'] = [0,1]

In [69]:
try_raw[5]

array([-3.14190838e-09, -1.01479415e-06, -2.20001439e-06, ...,
        3.36086342e-06,  2.76147522e-06,  1.70250471e-06])

In [67]:
#STILL PRACTICE
list_names = tryout.ch_names
columns = df.columns
for name in list_names:
  for nick in columns:
    if name == nick:
      pos = list_names.index(name)
      #df.at[LINE IN DF, nick] = try_raw[pos]



SyntaxError: ignored

In [ ]:
df_columns = data_files.columns #columns in the final data frame
def in_the_dataframe(file):
  timeseries = preprocessing(file)
  data = timeseries.get_data() #array file of certain shape (21 channels after drop out)
  channel_name = timeseries.ch_names
  for name in channel_name:
    for nick in df_columns:
      if name == nick:
        pos = list_names.index(name)
        #data_files.at[LINE IN DF, nick] = data[pos]

#Basically, you get the array like data from preprocessed edf file of shape [number of channels, number of time points],
#add each raw of the array file (which corresponds to different channels) as a list to the corresponding cell
#in the final data frame. BUT we also need to allign a particular    


In [77]:
'''data_first = preprocessing(file[0])
df_first = pd.DataFrame(data_first.get_data())
df_first['channel'] = data_first.ch_names
df_first['patient_id'] = file[0][-22:-14]
for i in range(1, len(file)):
  data = preprocessing(file[i])
  df = data.get_data()
  df = pd.DataFrame(df)
  df['channel'] = data.ch_names
  df['patient_id'] = file[i][-22:-14]
  df_first = pd.concat([df_first, df])'''

TypeError: ignored

In [ ]:
df_first.shape

(96, 44802)

In [ ]:
df_first.columns

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       44790, 44791, 44792, 44793, 44794, 44795, 44796, 44797, 44798, 44799],
      dtype='object', length=44802)

In [ ]:
#DOESN'T WORK PROPERLY
data_first = preprocessing(file[0])
df_first = pd.DataFrame(data_first.get_data())
df_first['channel'] = data_first.ch_names
df_first['patient_id'] = file[0][-22:-14]
baseline_samples = data_first.n_times
for i in range(1, len(file)):
  data = preprocessing(file[i])
  df = data.get_data()
  '''if data.n_times > baseline_samples:
    counter = 0
    for j in range(0, data.n_times, baseline_samples):
      if j != 0:
        df1 = pd.DataFrame(df[counter:j])
        print(df1.shape)
        print(len(data.ch_names))
        print(df_first.shape)
        df1['channel'] = data.ch_names
        df1['patient_id'] = file[i][-22:-14]
        df_first = pd.concat([df_first, df1])
        counter += j'''
  else:
    df = pd.DataFrame(df)
    df['channel'] = data.ch_names
    df['patient_id'] = file[i][-22:-14]
    df_first = pd.concat([df_first, df])

Extracting EDF parameters from /content/drive/MyDrive/MLTSA22/Final_Project/00000258_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7999  =      0.000 ...    19.997 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB c